In [2]:
import cv2
import numpy as np
import math
import glob
from matplotlib import pyplot as plt
from IPython.display import Image, display

In [89]:
for filename in sorted(glob.glob('data_set/men/0/*.JPG')):
    img = cv2.imread(filename) 
    img = cv2.cvtColor(img,cv2.COLOR_BGR2YCrCb)
    img_masked = cv2.inRange(img,(0,137,77),(255,163,120))
    cv2.imwrite("masks/"+filename,img_masked)
# plt.imshow(img_masked)
# np.array((0,137,77),dtype=np.uint8)
# img_masked

In [82]:
img = cv2.cvtColor(cv2.imread("data_set/men/3/3_men (5).JPG"),cv2.COLOR_BGR2YCrCb)
img_masked = cv2.inRange(img,(0,137,77),(255,163,140))
cv2.imwrite("masked.JPG",img_masked)
# np.array((0,137,77),dtype=np.uint8)
# img_masked


True

In [ ]:
import cv2 as cv
import numpy as np
from matplotlib import pyplot as plt
img = cv.imread('data_set/men/3/3_men (5).JPG', cv.IMREAD_GRAYSCALE)
assert img is not None, "file could not be read, check with os.path.exists()"
# global thresholding
ret1,th1 = cv.threshold(img,127,255,cv.THRESH_BINARY)
# Otsu's thresholding
ret2,th2 = cv.threshold(img,0,255,cv.THRESH_BINARY+cv.THRESH_OTSU)
# Otsu's thresholding after Gaussian filtering
blur = cv.GaussianBlur(img,(5,5),0)
ret3,th3 = cv.threshold(blur,0,255,cv.THRESH_BINARY+cv.THRESH_OTSU)
# plot all the images and their histograms
images = [img, 0, th1,
          img, 0, th2,
          blur, 0, th3]
titles = ['Original Noisy Image','Histogram','Global Thresholding (v=127)',
          'Original Noisy Image','Histogram',"Otsu's Thresholding",
          'Gaussian filtered Image','Histogram',"Otsu's Thresholding"]
for i in range(3):
    plt.subplot(3,3,i*3+1),plt.imshow(images[i*3],'gray')
    plt.title(titles[i*3]), plt.xticks([]), plt.yticks([])
    plt.subplot(3,3,i*3+2),plt.hist(images[i*3].ravel(),256)
    plt.title(titles[i*3+1]), plt.xticks([]), plt.yticks([])
    plt.subplot(3,3,i*3+3),plt.imshow(images[i*3+2],'gray')
    plt.title(titles[i*3+2]), plt.xticks([]), plt.yticks([])
plt.show()

In [4]:
import cv2
import mediapipe as mp
import time

# frameWidth = 1280
# frameHeight = 720
# cap = cv2.VideoCapture(0)
# cap.set(3, frameWidth)
# cap.set(4, frameHeight)
mpHands=mp.solutions.hands
hands=mpHands.Hands()
mpDraw = mp.solutions.drawing_utils

# pTime = 0
# cTime = 0
filename = 'data_set/men/3/3_men (5).JPG'
img = cv2.imread(filename)
img= cv2.flip(img,1)
img_b = np.empty(img.shape)
img_b.fill(0)
imgRGB = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
results = hands.process(imgRGB)
#print(results.multi_hand_landmarks)
if results.multi_hand_landmarks:
    for handLms in results.multi_hand_landmarks: 
    #handLMs are 21 points. so we need conection too-->mpHands.HAND_CONNECTIONS
        for id, lm in enumerate(handLms.landmark):
            #print(id, lm)
            #lm = x,y cordinate of each landmark in float numbers. lm.x, lm.y methods
            #So, need to covert in integer
            h, w, c =img.shape
            cx, cy = int(lm.x * w), int(lm.y * h)
            #print(id, cx, cy)
            # if id == 4: #(To draw 4th point)
            #cv2.circle(img, (cx, cy), 15, (255, 0, 255), cv2.FILLED)
        mpDraw.draw_landmarks(img_b, handLms, mpHands.HAND_CONNECTIONS) #drawing points and lines(=handconections)


cv2.imwrite('masked.JPG', img_b)

True